In [7]:
import json
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [1]:
import json
import pandas as pd
import os

def load_mouse_movement_data(file_paths):
    data = []
    for file_path in file_paths:
        with open(file_path) as f:
            movements = json.load(f)
            data.append(movements)
    return data

# Define your file paths here
train_human_paths = [
    "C:\\Users\\Nidhi\\OneDrive - SSN Trust\\Desktop\\SNU_CLG\\SIH\\phase1\\data\\mouse_movements\\humans_and_moderate_bots\\dr09rk5eagjuu87gedvdqmq3gl\\mouse_movements.json",
    "C:\\Users\\Nidhi\\OneDrive - SSN Trust\\Desktop\\SNU_CLG\\SIH\\phase1\\data\\mouse_movements\\humans_and_moderate_bots\\gq715ms79515gcq39vf91mli6t\\mouse_movements.json",
    "C:\\Users\\Nidhi\\OneDrive - SSN Trust\\Desktop\\SNU_CLG\\SIH\\phase1\\data\\mouse_movements\\humans_and_moderate_bots\\hrbko2t4t14q3pahqltndlolb5\\mouse_movements.json",
]

train_moderate_bot_paths = [
    "C:\\Users\\Nidhi\\OneDrive - SSN Trust\\Desktop\\SNU_CLG\\SIH\\phase1\\data\\mouse_movements\\humans_and_advanced_bots\\3uqepgd76f9ecnauehcl4sucbh\\mouse_movements.json",
    "C:\\Users\\Nidhi\\OneDrive - SSN Trust\\Desktop\\SNU_CLG\\SIH\\phase1\\data\\mouse_movements\\humans_and_advanced_bots\\ck0vis16184tm6572eohin19d2\\mouse_movements.json",
    "C:\\Users\\Nidhi\\OneDrive - SSN Trust\\Desktop\\SNU_CLG\\SIH\\phase1\\data\\mouse_movements\\humans_and_advanced_bots\\pf7tnis955pq27n6sibk32d87k\\mouse_movements.json",
]

train_human_data = load_mouse_movement_data(train_human_paths)
train_moderate_bot_data = load_mouse_movement_data(train_moderate_bot_paths)


C:\Users\Nidhi\AppData\Local\Temp\ipykernel_13120\3941224911.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
def extract_features(data, label):
    features = []
    for movements in data:
        # Adjust these keys based on your JSON structure
        if 'movements' in movements:  # Assuming your data might have a key called 'movements'
            mouse_data = movements['movements']  # Adjust based on your actual key name
            xs = [move['x'] for move in mouse_data]  # Change this if 'x' is nested differently
            ys = [move['y'] for move in mouse_data]
            times = [move['timestamp'] for move in mouse_data]  # Adjust this key accordingly

            # Calculate speeds and other features
            speeds = [((xs[i] - xs[i-1])**2 + (ys[i] - ys[i-1])**2)**0.5 / (times[i] - times[i-1]) for i in range(1, len(xs))]
            features.append([len(xs), sum(speeds)/len(speeds) if speeds else 0, max(speeds, default=0), min(speeds, default=0), label])
    
    return features


In [13]:
# Step 1: Inspect the JSON Structure
def inspect_json_structure(file_paths):
    for file_path in file_paths:
        with open(file_path) as f:
            movements = json.load(f)
            print(f"Data from {file_path}:")
            print(movements)  # Print the entire structure
            print("\nSample Movement Data:")
            if 'movements' in movements:
                print(movements['movements'][:5])  # Print the first 5 movement records for inspection
            else:
                print("No 'movements' key found in this JSON.\n")
            print("="*50)

# Run the inspection on the training data
inspect_json_structure(train_human_paths)
inspect_json_structure(train_moderate_bot_paths)


Data from C:\Users\Nidhi\OneDrive - SSN Trust\Desktop\SNU_CLG\SIH\phase1\data\mouse_movements\humans_and_moderate_bots\dr09rk5eagjuu87gedvdqmq3gl\mouse_movements.json:
{'session_id': 'dr09rk5eagjuu87gedvdqmq3gl', 'total_behaviour': '[m(246,6)][m(244,9)][m(239,18)][m(235,26)][m(231,35)][m(223,43)][m(215,50)][m(208,55)][m(197,63)][m(189,70)][m(181,82)][m(176,95)][m(172,109)][m(172,129)][m(174,143)][m(178,157)][m(180,163)][m(182,168)][m(185,171)][m(188,174)][m(192,175)][m(195,176)][m(199,176)][m(202,177)][m(210,179)][m(221,183)][m(226,183)][m(237,187)][m(241,187)][m(244,187)][m(247,187)][m(249,186)][m(253,185)][m(259,182)][m(263,182)][m(267,180)][m(268,180)][m(269,180)][m(270,180)][m(275,182)][m(279,184)][m(287,187)][m(294,188)][m(304,191)][m(317,193)][m(329,194)][m(344,195)][m(361,195)][m(384,192)][m(397,191)][m(408,190)][m(421,190)][m(440,191)][m(462,194)][m(482,197)][m(501,200)][m(516,202)][m(524,202)][m(526,202)][m(527,202)][m(528,202)][m(529,202)][m(531,202)][m(532,202)][m(533,202)][

In [14]:
import re

def extract_coordinates(total_behaviour):
    # Regular expression to extract (x, y) coordinates from the string
    pattern = r'\[m\((\d+),(\d+)\)\]'
    matches = re.findall(pattern, total_behaviour)
    
    # Convert the extracted matches into lists of x and y coordinates
    xs = [int(match[0]) for match in matches]
    ys = [int(match[1]) for match in matches]
    
    return xs, ys

# Test the function with one of the files
for file_path in train_human_paths:
    with open(file_path) as f:
        movements = json.load(f)
        xs, ys = extract_coordinates(movements['total_behaviour'])
        print(f"Coordinates from {file_path}:")
        print(f"x: {xs[:5]}")
        print(f"y: {ys[:5]}")
        print("=" * 50)


Coordinates from C:\Users\Nidhi\OneDrive - SSN Trust\Desktop\SNU_CLG\SIH\phase1\data\mouse_movements\humans_and_moderate_bots\dr09rk5eagjuu87gedvdqmq3gl\mouse_movements.json:
x: [246, 244, 239, 235, 231]
y: [6, 9, 18, 26, 35]
Coordinates from C:\Users\Nidhi\OneDrive - SSN Trust\Desktop\SNU_CLG\SIH\phase1\data\mouse_movements\humans_and_moderate_bots\gq715ms79515gcq39vf91mli6t\mouse_movements.json:
x: [316, 315, 312, 311, 309]
y: [2, 4, 7, 9, 13]
Coordinates from C:\Users\Nidhi\OneDrive - SSN Trust\Desktop\SNU_CLG\SIH\phase1\data\mouse_movements\humans_and_moderate_bots\hrbko2t4t14q3pahqltndlolb5\mouse_movements.json:
x: [303, 303, 303, 303, 303]
y: [0, 1, 3, 5, 9]


In [15]:
import numpy as np

def calculate_features(xs, ys):
    # Calculate the Euclidean distance between consecutive points
    distances = [((xs[i] - xs[i-1])**2 + (ys[i] - ys[i-1])**2)**0.5 for i in range(1, len(xs))]
    
    # Speed can be considered as distance since we don't have time stamps
    total_distance = sum(distances)
    avg_speed = np.mean(distances) if distances else 0
    max_speed = max(distances) if distances else 0
    min_speed = min(distances) if distances else 0
    
    return total_distance, avg_speed, max_speed, min_speed

# Test the function on the same datasets
for file_path in train_human_paths:
    with open(file_path) as f:
        movements = json.load(f)
        xs, ys = extract_coordinates(movements['total_behaviour'])
        total_distance, avg_speed, max_speed, min_speed = calculate_features(xs, ys)
        print(f"Features for {file_path}:")
        print(f"Total Distance: {total_distance}")
        print(f"Average Speed: {avg_speed}")
        print(f"Max Speed: {max_speed}")
        print(f"Min Speed: {min_speed}")
        print("=" * 50)


Features for C:\Users\Nidhi\OneDrive - SSN Trust\Desktop\SNU_CLG\SIH\phase1\data\mouse_movements\humans_and_moderate_bots\dr09rk5eagjuu87gedvdqmq3gl\mouse_movements.json:
Total Distance: 49574.16932280646
Average Speed: 5.950566477350471
Max Speed: 679.0184091760694
Min Speed: 1.0
Features for C:\Users\Nidhi\OneDrive - SSN Trust\Desktop\SNU_CLG\SIH\phase1\data\mouse_movements\humans_and_moderate_bots\gq715ms79515gcq39vf91mli6t\mouse_movements.json:
Total Distance: 44201.01794084707
Average Speed: 7.337486377962702
Max Speed: 644.8108249711694
Min Speed: 1.0
Features for C:\Users\Nidhi\OneDrive - SSN Trust\Desktop\SNU_CLG\SIH\phase1\data\mouse_movements\humans_and_moderate_bots\hrbko2t4t14q3pahqltndlolb5\mouse_movements.json:
Total Distance: 53076.27908171755
Average Speed: 6.1580553523282955
Max Speed: 270.44038160008574
Min Speed: 1.0


In [16]:
# Prepare labels (1 for humans, 0 for moderate bots)
human_labels = [1] * len(train_human_data)
moderate_bot_labels = [0] * len(train_moderate_bot_data)

# Extract features for all data
def extract_all_features(file_paths, label):
    features_list = []
    for file_path in file_paths:
        with open(file_path) as f:
            movements = json.load(f)
            xs, ys = extract_coordinates(movements['total_behaviour'])
            total_distance, avg_speed, max_speed, min_speed = calculate_features(xs, ys)
            features_list.append([total_distance, avg_speed, max_speed, min_speed, label])
    return features_list

# Extract features for human and bot data
human_features = extract_all_features(train_human_paths, 1)  # Label 1 for humans
moderate_bot_features = extract_all_features(train_moderate_bot_paths, 0)  # Label 0 for bots

# Combine features
data = human_features + moderate_bot_features

# Create a DataFrame
columns = ['Total Distance', 'Average Speed', 'Max Speed', 'Min Speed', 'Label']
df = pd.DataFrame(data, columns=columns)

# Show the DataFrame
print(df.head())


   Total Distance  Average Speed   Max Speed  Min Speed  Label
0    49574.169323       5.950566  679.018409        1.0      1
1    44201.017941       7.337486  644.810825        1.0      1
2    53076.279082       6.158055  270.440382        1.0      1
3    91971.473548      10.052626  353.000000        0.0      0
4    78576.825966      10.348588  625.012800        0.0      0


In [19]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(X_train_scaled, y_train)
y_pred = clf.predict(X_test_scaled)


In [20]:
# Split the data into features (X) and labels (y)
X = df.drop('Label', axis=1)
y = df['Label']

# Split into training and testing sets (70% training, 30% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Normalize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train a RandomForestClassifier
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = clf.predict(X_test_scaled)

# Print the classification report to evaluate the model
print(classification_report(y_test, y_pred))

# Feature Importance (optional, but useful to know what features are most relevant)
importances = clf.feature_importances_
for feature, importance in zip(X.columns, importances):
    print(f'Feature: {feature}, Importance: {importance}')


              precision    recall  f1-score   support

           0       0.00      0.00      0.00       0.0
           1       0.00      0.00      0.00       2.0

    accuracy                           0.00       2.0
   macro avg       0.00      0.00      0.00       2.0
weighted avg       0.00      0.00      0.00       2.0

Feature: Total Distance, Importance: 0.24193548387096775
Feature: Average Speed, Importance: 0.3225806451612903
Feature: Max Speed, Importance: 0.22580645161290322
Feature: Min Speed, Importance: 0.20967741935483872


C:\Users\Nidhi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Nidhi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Nidhi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classification.py:1497: Un

In [21]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Creating and training the SVM Classifier
svm_model = SVC(kernel='linear', random_state=42)  # You can try other kernels as well
svm_model.fit(X_train, y_train)

# Making predictions
y_pred = svm_model.predict(X_test)

# Evaluating the model
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           1       1.00      1.00      1.00         2

    accuracy                           1.00         2
   macro avg       1.00      1.00      1.00         2
weighted avg       1.00      1.00      1.00         2



In [22]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Creating and training the Gradient Boosting Classifier
gb_model = GradientBoostingClassifier(random_state=42)
gb_model.fit(X_train, y_train)

# Making predictions
y_pred = gb_model.predict(X_test)

# Evaluating the model
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           1       1.00      1.00      1.00         2

    accuracy                           1.00         2
   macro avg       1.00      1.00      1.00         2
weighted avg       1.00      1.00      1.00         2



In [28]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from joblib import dump

# Split your data (X and y) into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the Gradient Boosting Classifier
gb_model = GradientBoostingClassifier(random_state=42)
gb_model.fit(X_train, y_train)

# Save the model to a file
dump(gb_model, 'gradient_boosting_model.joblib')

print("Model saved successfully!")


Model saved successfully!


In [30]:
from joblib import load
import numpy as np

# Load the saved model
gb_model = load('gradient_boosting_model.joblib')

# Function to predict whether the data represents a human or bot
def predict_movement(coordinates):
    # Assuming coordinates is a 2D array-like structure
    # For example: coordinates = [[x1, y1], [x2, y2], ...]
    
    # Extract features from the coordinates
    # Example: calculate total distance, average speed, etc.
    total_distance = np.sum(np.sqrt(np.sum(np.diff(coordinates, axis=0) ** 2, axis=1)))  # Calculate total distance
    speeds = np.linalg.norm(np.diff(coordinates, axis=0), axis=1)  # Calculate speeds
    average_speed = np.mean(speeds)
    max_speed = np.max(speeds)
    min_speed = np.min(speeds)
    
    # Prepare the feature vector for prediction
    features = np.array([[total_distance, average_speed, max_speed, min_speed]])  # Adjust as necessary

    # Make predictions
    prediction = gb_model.predict(features)
    
    return prediction

# Example usage
# Replace the following coordinates with your actual test data
test_coordinates = [[100, 200], [150, 250], [200, 300], [250, 350]]  # Example coordinates
prediction = predict_movement(test_coordinates)
print("Prediction (0 for Human, 1 for Bot):", prediction[0])


Prediction (0 for Human, 1 for Bot): 1


C:\Users\Nidhi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(
